In [32]:
# ============================================================================
# TAHAP 1B: MANUAL COLUMN MAPPING & DEEP CLEANING
# PT Arkonin Engineering Manggala Pratama - Tender Prediction System
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("=" * 80)
print("TAHAP 1B: MANUAL COLUMN MAPPING & DEEP CLEANING")
print("=" * 80)

# ============================================================================
# 1. LOAD DATA DENGAN INSPEKSI MANUAL
# ============================================================================
print("\n[1] Loading dan Inspeksi Struktur Excel...")

file_path = "PP PT ARKONIN EMP Thn 2022 sd 2025.xlsx"

# Baca raw tanpa header
df_raw = pd.read_excel(file_path, sheet_name=0, header=None)

print(f"\n✓ Data dimuat: {df_raw.shape[0]} baris × {df_raw.shape[1]} kolom")

# Tampilkan 15 baris pertama untuk inspeksi
print("\n📋 Inspeksi 15 Baris Pertama:")
print(df_raw.head(15))

TAHAP 1B: MANUAL COLUMN MAPPING & DEEP CLEANING

[1] Loading dan Inspeksi Struktur Excel...

✓ Data dimuat: 270 baris × 38 kolom

📋 Inspeksi 15 Baris Pertama:
     0                                                  1                     2                                                  3   4   5   6   7   8                            9          10  11      12                     13  14             15                                                 16  17  18  19  20  21      22       23  24            25                                         26  27           28            29  30                    31         32  33                   34   35  36   37
0   NaN  DATA PENGALAMAN PERUSAHAAN PT. ARKONIN ENGINEE...                   NaN                                                NaN NaN NaN NaN NaN NaN                          NaN        NaN NaN     NaN                    NaN NaN            NaN                                                NaN NaN NaN NaN NaN NaN     NaN      NaN NaN  

In [33]:
# ============================================================================
# 2. IDENTIFIKASI HEADER ROW SECARA MANUAL
# ============================================================================
print("\n[2] Identifikasi Header Row...")
print("-" * 80)

# Cari baris yang mengandung keyword kunci
header_keywords = ['nama', 'paket', 'pekerjaan', 'klasifikasi', 'lokasi', 
                   'pengguna', 'jasa', 'kontrak', 'nilai', 'tanggal']

potential_header_rows = []

for idx in range(min(10, len(df_raw))):
    row_text = ' '.join([str(val).lower() for val in df_raw.iloc[idx] if pd.notna(val)])
    keyword_count = sum(1 for keyword in header_keywords if keyword in row_text)
    
    if keyword_count >= 3:  # Minimal 3 keyword cocok
        potential_header_rows.append((idx, keyword_count))
        print(f"  Row {idx}: {keyword_count} keywords found")

if potential_header_rows:
    # Pilih row dengan keyword terbanyak
    header_row = max(potential_header_rows, key=lambda x: x[1])[0]
    print(f"\n✓ Header row terdeteksi: Baris {header_row}")
else:
    header_row = 2  # Default fallback
    print(f"\n⚠️ Header row tidak terdeteksi, menggunakan default: Baris {header_row}")

# ============================================================================
# 3. EKSTRAK HEADER DAN DATA
# ============================================================================
print("\n[3] Ekstrak Header dan Data...")
print("-" * 80)

# Ambil header dari beberapa baris (untuk merged cells)
header_rows = []
for i in range(header_row, min(header_row + 3, len(df_raw))):
    header_rows.append(df_raw.iloc[i])

# Gabungkan header dari multiple rows
column_names = []
for col_idx in range(len(df_raw.columns)):
    col_parts = []
    for header in header_rows:
        val = header.iloc[col_idx]
        if pd.notna(val) and str(val).strip() != '':
            col_parts.append(str(val).strip())
    
    if col_parts:
        column_name = ' - '.join(col_parts)
    else:
        column_name = f'Unnamed_{col_idx}'
    
    column_names.append(column_name)

print(f"\n✓ Header ekstrak berhasil: {len(column_names)} kolom")
print("\n📝 Nama Kolom Hasil Ekstraksi:")
for idx, name in enumerate(column_names, 1):
    print(f"  {idx:2d}. {name}")



[2] Identifikasi Header Row...
--------------------------------------------------------------------------------
  Row 2: 5 keywords found
  Row 3: 3 keywords found
  Row 4: 4 keywords found

✓ Header row terdeteksi: Baris 2

[3] Ekstrak Header dan Data...
--------------------------------------------------------------------------------

✓ Header ekstrak berhasil: 38 kolom

📝 Nama Kolom Hasil Ekstraksi:
   1. No.
   2. Unnamed_1
   3. Nama Paket Pekerjaan
   4. Unnamed_3
   5. Unnamed_4
   6. Unnamed_5
   7. Unnamed_6
   8. Unnamed_7
   9. Unnamed_8
  10. Klasifikasi/Sub Klasifikasi
  11. Unnamed_10
  12. Unnamed_11
  13. Lokasi
  14. Unnamed_13
  15. Unnamed_14
  16. Pengguna Jasa - Nama
  17. Unnamed_16
  18. Unnamed_17
  19. Unnamed_18
  20. Unnamed_19
  21. Unnamed_20
  22. Unnamed_21
  23. Alamat
  24. Unnamed_23
  25. Unnamed_24
  26. Kontrak - No./Tanggal
  27. Unnamed_26
  28. Unnamed_27
  29. Nilai (Rp.)
  30. Unnamed_29
  31. Unnamed_30
  32. Tgl. Selesai Menurut - Kontrak
  3

In [34]:
# ============================================================================
# 4. CREATE CLEAN DATAFRAME
# ============================================================================
print("\n[4] Create Clean DataFrame...")
print("-" * 80)

# Ambil data mulai dari baris setelah header
data_start_row = header_row + 3
df_clean = df_raw.iloc[data_start_row:].copy()
df_clean.columns = column_names
df_clean = df_clean.reset_index(drop=True)

# Remove baris yang semua kolom penting-nya kosong
important_cols = [col for col in df_clean.columns if 'Unnamed' not in col]
if important_cols:
    df_clean = df_clean.dropna(subset=important_cols, how='all')

print(f"\n✓ DataFrame bersih dibuat:")
print(f"  - Baris: {len(df_clean)}")
print(f"  - Kolom: {len(df_clean.columns)}")

# ============================================================================
# 5. MANUAL COLUMN MAPPING BERDASARKAN KEBUTUHAN
# ============================================================================
print("\n[5] Manual Column Mapping...")
print("-" * 80)

# Mapping ke nama kolom standar
column_mapping = {}

# Cari kolom untuk setiap kebutuhan
target_columns = {
    'Sektor': ['sektor', 'bidang'],
    'Nama_Paket': ['nama', 'paket', 'pekerjaan', 'project'],
    'Klasifikasi': ['klasifikasi', 'jenis', 'kategori'],
    'Sub_Klasifikasi': ['sub', 'sub klasifikasi', 'sub-klasifikasi'],
    'Lokasi': ['lokasi', 'tempat', 'kota', 'daerah'],
    'Pengguna_Jasa_Nama': ['pengguna jasa', 'klien', 'owner', 'pemberi'],
    'Pengguna_Jasa_Alamat': ['alamat', 'address'],
    'Kontrak_Nomor': ['no kontrak', 'nomor kontrak', 'no. kontrak'],
    'Kontrak_Tanggal': ['tanggal kontrak', 'tgl kontrak'],
    'Nilai_Kontrak': ['nilai', 'harga', 'rp', 'rupiah'],
    'Tanggal_Mulai': ['mulai', 'start', 'awal'],
    'Tanggal_Selesai_Kontrak': ['selesai', 'finish', 'akhir', 'kontrak'],
    'Tanggal_Selesai_BA': ['ba', 'berita acara', 'lap akhir']
}

for target_name, keywords in target_columns.items():
    for col in df_clean.columns:
        col_lower = str(col).lower()
        if any(keyword in col_lower for keyword in keywords):
            column_mapping[target_name] = col
            break

print("\n✓ Kolom Berhasil di-mapping:")
for target, actual in column_mapping.items():
    print(f"  • {target:25s} → {actual}")

# Kolom yang belum terdeteksi
missing_targets = [t for t in target_columns.keys() if t not in column_mapping]
if missing_targets:
    print("\n⚠️ Kolom yang BELUM terdeteksi:")
    for target in missing_targets:
        print(f"  ❌ {target}")



[4] Create Clean DataFrame...
--------------------------------------------------------------------------------

✓ DataFrame bersih dibuat:
  - Baris: 60
  - Kolom: 38

[5] Manual Column Mapping...
--------------------------------------------------------------------------------

✓ Kolom Berhasil di-mapping:
  • Nama_Paket                → Nama Paket Pekerjaan
  • Klasifikasi               → Klasifikasi/Sub Klasifikasi
  • Sub_Klasifikasi           → Klasifikasi/Sub Klasifikasi
  • Lokasi                    → Lokasi
  • Pengguna_Jasa_Nama        → Pengguna Jasa - Nama
  • Pengguna_Jasa_Alamat      → Alamat
  • Nilai_Kontrak             → Nilai (Rp.)
  • Tanggal_Selesai_Kontrak   → Kontrak - No./Tanggal
  • Tanggal_Selesai_BA        → BA Peny. Lap. Akhir

⚠️ Kolom yang BELUM terdeteksi:
  ❌ Sektor
  ❌ Kontrak_Nomor
  ❌ Kontrak_Tanggal
  ❌ Tanggal_Mulai


In [35]:
# 6. CREATE FINAL DATAFRAME DENGAN KOLOM STANDAR
# ============================================================================
print("\n[6] Create Final DataFrame...")
print("-" * 80)

# Buat dataframe baru dengan nama kolom standar
df_final = pd.DataFrame()

for target_name, actual_col in column_mapping.items():
    if actual_col in df_clean.columns:
        df_final[target_name] = df_clean[actual_col]

print(f"\n✓ Final DataFrame:")
print(f"  - Baris: {len(df_final)}")
print(f"  - Kolom: {len(df_final.columns)}")



[6] Create Final DataFrame...
--------------------------------------------------------------------------------

✓ Final DataFrame:
  - Baris: 60
  - Kolom: 9


In [37]:
# ============================================================================
# 7. DATA TYPE CONVERSION
# ============================================================================
print("\n[7] Data Type Conversion...")
print("-" * 80)

# Konversi tipe data
if 'Nilai_Kontrak' in df_final.columns:
    # Clean currency format
    df_final['Nilai_Kontrak'] = df_final['Nilai_Kontrak'].astype(str).str.replace(',', '').str.replace('.', '')
    df_final['Nilai_Kontrak'] = pd.to_numeric(df_final['Nilai_Kontrak'], errors='coerce')
    print("✓ Nilai_Kontrak → numeric")

# Konversi tanggal
date_columns = [col for col in df_final.columns if 'Tanggal' in col]
for col in date_columns:
    df_final[col] = pd.to_datetime(df_final[col], errors='coerce')
    print(f"✓ {col} → datetime")



[7] Data Type Conversion...
--------------------------------------------------------------------------------
✓ Nilai_Kontrak → numeric
✓ Tanggal_Selesai_Kontrak → datetime
✓ Tanggal_Selesai_BA → datetime


In [38]:
# ============================================================================
# 8. DATA QUALITY REPORT
# ============================================================================
print("\n[8] Data Quality Report...")
print("-" * 80)

print("\n📊 Missing Values per Kolom:")
missing_summary = pd.DataFrame({
    'Kolom': df_final.columns,
    'Missing': df_final.isnull().sum(),
    'Persen': (df_final.isnull().sum() / len(df_final) * 100).round(2)
})
print(missing_summary.to_string(index=False))

print("\n📊 Data Type Summary:")
print(df_final.dtypes)

print("\n📊 Sample Data (5 baris pertama):")
print(df_final.head())



[8] Data Quality Report...
--------------------------------------------------------------------------------

📊 Missing Values per Kolom:
                  Kolom  Missing  Persen
             Nama_Paket       59   98.33
            Klasifikasi       59   98.33
        Sub_Klasifikasi       59   98.33
                 Lokasi       59   98.33
     Pengguna_Jasa_Nama       59   98.33
   Pengguna_Jasa_Alamat       59   98.33
          Nilai_Kontrak       59   98.33
Tanggal_Selesai_Kontrak       60  100.00
     Tanggal_Selesai_BA       60  100.00

📊 Data Type Summary:
Nama_Paket                         object
Klasifikasi                        object
Sub_Klasifikasi                    object
Lokasi                             object
Pengguna_Jasa_Nama                 object
Pengguna_Jasa_Alamat               object
Nilai_Kontrak                     float64
Tanggal_Selesai_Kontrak    datetime64[ns]
Tanggal_Selesai_BA         datetime64[ns]
dtype: object

📊 Sample Data (5 baris pertama):
   N

In [39]:
# ============================================================================
# 9. EXPORT HASIL FINAL
# ============================================================================
print("\n[9] Export Hasil Final...")
print("-" * 80)

# Export to Excel
df_final.to_excel('02_data_final_cleaned.xlsx', index=False)
print("✓ Final data → 02_data_final_cleaned.xlsx")

# Export to CSV (untuk backup)
df_final.to_csv('02_data_final_cleaned.csv', index=False)
print("✓ Final data → 02_data_final_cleaned.csv")

# Export to Pickle (untuk Python)
df_final.to_pickle('02_data_final_cleaned.pkl')
print("✓ Final data → 02_data_final_cleaned.pkl")

# Column mapping documentation
with open('02_column_mapping_final.txt', 'w', encoding='utf-8') as f:
    f.write("=" * 80 + "\n")
    f.write("FINAL COLUMN MAPPING\n")
    f.write("=" * 80 + "\n\n")
    f.write("Kolom Standar          → Kolom Original\n")
    f.write("-" * 80 + "\n")
    for target, actual in column_mapping.items():
        f.write(f"{target:25s} → {actual}\n")
    
    if missing_targets:
        f.write("\n" + "=" * 80 + "\n")
        f.write("KOLOM YANG BELUM TERDETEKSI\n")
        f.write("=" * 80 + "\n")
        for target in missing_targets:
            f.write(f"❌ {target}\n")

print("✓ Mapping docs → 02_column_mapping_final.txt")



[9] Export Hasil Final...
--------------------------------------------------------------------------------
✓ Final data → 02_data_final_cleaned.xlsx
✓ Final data → 02_data_final_cleaned.csv
✓ Final data → 02_data_final_cleaned.pkl
✓ Mapping docs → 02_column_mapping_final.txt


In [40]:
# ============================================================================
# 10. FINAL SUMMARY
# ============================================================================
summary = f"""
{'=' * 80}
FINAL SUMMARY - DATA CLEANING PROCESS
{'=' * 80}

📁 FILE INFO
- Source: {file_path}
- Header Row: {header_row}
- Data Start Row: {data_start_row}

📊 DATA DIMENSIONS
- Total Records: {len(df_final)}
- Total Columns: {len(df_final.columns)}
- Mapped Columns: {len(column_mapping)}
- Missing Targets: {len(missing_targets)}

📈 DATA QUALITY
- Total Cells: {len(df_final) * len(df_final.columns)}
- Missing Cells: {df_final.isnull().sum().sum()}
- Completeness: {((1 - df_final.isnull().sum().sum() / (len(df_final) * len(df_final.columns))) * 100):.2f}%

✅ BERHASIL DI-MAPPING:
{chr(10).join([f'   • {k}' for k in column_mapping.keys()])}

{'❌ BELUM TERDETEKSI:' if missing_targets else ''}
{chr(10).join([f'   • {t}' for t in missing_targets]) if missing_targets else ''}

{'=' * 80}
"""

print(summary)

with open('02_final_summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print("\n✓ Summary → 02_final_summary.txt")

print("\n" + "=" * 80)
print("✓ TAHAP 1B SELESAI!")
print("=" * 80)
print("\nFile yang dihasilkan:")
print("  1. 02_data_final_cleaned.xlsx")
print("  2. 02_data_final_cleaned.csv")
print("  3. 02_data_final_cleaned.pkl")
print("  4. 02_column_mapping_final.txt")
print("  5. 02_final_summary.txt")
print("\n🎯 Ready untuk TAHAP 2: EDA & Feature Engineering!")


FINAL SUMMARY - DATA CLEANING PROCESS

📁 FILE INFO
- Source: PP PT ARKONIN EMP Thn 2022 sd 2025.xlsx
- Header Row: 2
- Data Start Row: 5

📊 DATA DIMENSIONS
- Total Records: 60
- Total Columns: 9
- Mapped Columns: 9
- Missing Targets: 4

📈 DATA QUALITY
- Total Cells: 540
- Missing Cells: 533
- Completeness: 1.30%

✅ BERHASIL DI-MAPPING:
   • Nama_Paket
   • Klasifikasi
   • Sub_Klasifikasi
   • Lokasi
   • Pengguna_Jasa_Nama
   • Pengguna_Jasa_Alamat
   • Nilai_Kontrak
   • Tanggal_Selesai_Kontrak
   • Tanggal_Selesai_BA

❌ BELUM TERDETEKSI:
   • Sektor
   • Kontrak_Nomor
   • Kontrak_Tanggal
   • Tanggal_Mulai



✓ Summary → 02_final_summary.txt

✓ TAHAP 1B SELESAI!

File yang dihasilkan:
  1. 02_data_final_cleaned.xlsx
  2. 02_data_final_cleaned.csv
  3. 02_data_final_cleaned.pkl
  4. 02_column_mapping_final.txt
  5. 02_final_summary.txt

🎯 Ready untuk TAHAP 2: EDA & Feature Engineering!
